In [ ]:
!pip install -U --no-build-isolation --no-deps ../input/transformers-master/ -qq

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import os
import gc
gc.enable()
import math
import json
import time
import random
import multiprocessing
import pickle
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from sklearn import model_selection

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import (
    Dataset, DataLoader,
    SequentialSampler, RandomSampler
)
import matplotlib.pyplot as plt
import transformers
#from transformers import AutoTokenizer
from glob import glob
import pytorch_lightning as pl
from tqdm import tqdm
tqdm.pandas()
from pathlib import Path
import collections
import seaborn as sns
import gc


in_folder_path = Path('.')
train_scripts_dir = Path(in_folder_path)


In [ ]:
SINGLE_FOLD_MODE = True

In [ ]:
class ConfigTemplate:
    def __init__(self, config):
        self.__dict__.update(config)
   

In [ ]:
test_df = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
test_df = test_df.rename(columns={'id': 'index'})
test_df.head()

In [ ]:
from collections import defaultdict 
import gc

def postprocess_qa_predictions_new(data_df, archs_features_files, archs_preds_files, archs_weights):
    import matplotlib.pyplot as plt
    plt.rcParams["figure.figsize"] = (30, 15)
    
    DEBUG = False
    ####################
    START, END = 0, 1
    max_answer_length = 80

    archs_seqs = defaultdict(list)
    
    for arch_i, (arch_feats_file, arch_preds_file) in enumerate(zip(archs_feats_files, archs_preds_files)):
        with open(arch_feats_file, 'rb') as f:
            arch_feats = pickle.load(f)
        
        with open(arch_preds_file, 'rb') as f:
            arch_preds = pickle.load(f) 
            
        for _, row in data_df.iterrows():
            sample_index = row['index']
            context = row['context']
            char_start_logits = np.zeros(len(context))
            char_end_logits = np.zeros(len(context))

            arch_char_start_logits = np.zeros(len(context))
            arch_char_end_logits = np.zeros(len(context))
            
            for model_i, model_preds in enumerate(arch_preds):
                model_sample_preds = model_preds[sample_index]
                sample_features = arch_feats.query('index == @sample_index')
                if len(sample_features) == 0:
                    continue
                offsets_mappings = sample_features.offset_mapping.values
                sequences_ids = sample_features.sequence_ids.values

                model_char_start_logits = np.zeros(len(context))
                model_char_end_logits = np.zeros(len(context))
                offsets_counter = collections.Counter()

                for feat_i, (model_pred, sequence_ids, offsets_mapping) in enumerate(zip(model_sample_preds, sequences_ids, offsets_mappings)):
                    sequence_ids = np.array(sequence_ids)
                    offsets_mapping = np.array(offsets_mapping)

                    context_offsets_mapping = offsets_mapping[sequence_ids==1]
                    context_start_logits = np.array(model_pred[START])[sequence_ids==1]
                    context_end_logits = np.array(model_pred[END])[sequence_ids==1]

                    for (s, e), s_logit, e_logit in zip(context_offsets_mapping, context_start_logits, context_end_logits):
                        if offsets_counter[(s, e)]:
                            s_logit = max([s_logit, model_char_start_logits[s:e][0]])
                            e_logit = max([e_logit,  model_char_end_logits[s:e][0]])

                        model_char_start_logits[s:e] = s_logit
                        model_char_end_logits[s:e] = e_logit
                        offsets_counter[(s, e)] += 1

                arch_char_start_logits += model_char_start_logits
                arch_char_end_logits += model_char_end_logits
            
            #### AVERAGE arch_char_start_logits and arch_char_end_logits FIRST AND THEN APPLY WEIGHTS
            arch_char_start_logits = arch_char_start_logits * archs_weights[arch_i]
            arch_char_end_logits = arch_char_end_logits * archs_weights[arch_i]
            
            if not sample_index in archs_seqs:
                archs_seqs[sample_index] = (arch_char_start_logits, arch_char_end_logits)
            else:
                start_logits, end_logits = archs_seqs[sample_index]
                archs_seqs[sample_index] = (start_logits+arch_char_start_logits, end_logits+arch_char_end_logits)
                
        del arch_feats, arch_preds
        gc.collect()
            
    predictions = collections.OrderedDict()  
    for _, row in data_df.iterrows():
        sample_index = row['index']
        context = row['context']
        
        char_start_logits, char_end_logits = archs_seqs[sample_index]

        char_start_logits_indices = (char_start_logits > 0).nonzero()[0]
        char_start_logits = char_start_logits[char_start_logits_indices]

        char_end_logits_indices = (char_end_logits > 0).nonzero()[0]
        char_end_logits = char_end_logits[char_end_logits_indices]

        if DEBUG:
            plt.bar(range(char_start_logits.shape[0]), char_start_logits, color='black', edgecolor='black', )  
            plt.xticks(range(char_start_logits.shape[0]), char_start_logits_indices, rotation=-20)

            plt.show()

            plt.bar(range(char_end_logits.shape[0]), char_end_logits, color='black', edgecolor='black')    
            plt.xticks(range(char_end_logits.shape[0]), char_end_logits_indices, rotation=-20, )

            plt.show()


        inds = np.sort(np.unique(char_start_logits, return_index=True)[1])
        char_start_logits_unique = char_start_logits[inds]

        char_start_logits_indices_groups = np.split(char_start_logits_indices, inds)[1:]
        char_start_logits_indices_groups = [arr[0] for arr in char_start_logits_indices_groups]


        inds = np.sort(np.unique(char_end_logits, return_index=True)[1])
        char_end_logits_unique = char_end_logits[inds]

        char_end_logits_indices_groups = np.split(char_end_logits_indices, inds)[1:]

        char_end_logits_indices_groups = [arr[-1] for arr in char_end_logits_indices_groups]

        ####### get best probs
        valid_answers = []
        start_logits_to_indices = sorted(list(zip(char_start_logits_unique, char_start_logits_indices_groups)), reverse=True)
        end_logits_to_indices = sorted(list(zip(char_end_logits_unique, char_end_logits_indices_groups)), reverse=True)

        for start_logit, start_index in start_logits_to_indices:
            for end_logit, end_index in end_logits_to_indices:

                if (end_index < start_index or end_index - start_index + 1 > max_answer_length):
                    continue
                valid_answers.append(
                    {
                        'score': start_logit + end_logit,
                        'text': context[start_index: end_index+1].strip(),
                        'token_positions': (start_index, end_index),
                        'logits': (start_logit, end_logit)
                    }
                )    
        if len(valid_answers) > 0:
            best_answers = sorted(valid_answers, key=lambda x: x['score'], reverse=True)

            if DEBUG:
                for ans in best_answers[:10]:
                    print(ans)

            best_answer = best_answers[0]
        else:
            best_answer = {'text': '', 'score': 0.0}
        predictions[row['index']] = best_answer

    return predictions

In [ ]:
def normalize(v):
    temp = []
    for dat in v:
        norm = np.linalg.norm(dat)
        if norm == 0: 
            temp.append(dat)
        temp.append(dat/norm)
    return temp

In [ ]:
def normalize_v2(v):
    temp2 = []
    for dat in v:
        temp = []
        for d in dat:
            norm = np.linalg.norm(d)
            if norm == 0: 
                temp.append(d)
            temp.append(d/norm)
        temp2.append(temp)
    return temp2

In [ ]:
class FeatureExtractor:
    OUT_COLUMNS = ['index', 'input_ids', 'attention_mask', 'offset_mapping', 'sequence_ids']
    TRAIN_SPECIFIC_COLUMNS = ['start_position', 'end_position', 'kfold']
   
    
    def __init__(self, tokenizer, cls_token, test=False):
        self.tokenizer = tokenizer
        self.test = test
        self.columns = self.OUT_COLUMNS + [self.TRAIN_SPECIFIC_COLUMNS, []][self.test==True]
#         self.cls_token = '<CLS>'
        self.cls_token = cls_token
        self.cls_token_id = self.tokenizer.encode(self.cls_token)[0]
        
    def extract_features(self, df):
        features = []
        for i in tqdm(range(len(df))):
            row = df.iloc[i]
            tokenized_sample = self.tokenizer(
                row["question"].strip(),
                row["context"],
                truncation="only_second",
                max_length=Config.max_seq_length,
                stride=Config.doc_stride,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                padding="max_length",
            )


            sample_mapping = tokenized_sample.pop("overflow_to_sample_mapping")
            offset_mapping = tokenized_sample.pop("offset_mapping")
            
#             if not isinstance(answers["text"][0], str):
#                 continue

            for i, offsets in enumerate(offset_mapping):
                feature = {}

                input_ids = tokenized_sample["input_ids"][i]
                attention_mask = tokenized_sample["attention_mask"][i]
                feature['index'] = row['index']
                feature['input_ids'] = input_ids
                feature['attention_mask'] = attention_mask
                feature['offset_mapping'] = offsets
                feature['sequence_ids'] = [0 if i is None else i for i in tokenized_sample.sequence_ids(i)]
                if not self.test:
                    feature['kfold'] = row['kfold']
                    cls_index = input_ids.index(self.cls_token_id)
                    sequence_ids = tokenized_sample.sequence_ids(i)
                    answers = row["answers"]

                    sample_index = sample_mapping[i]
                    answers = row["answers"]

                    if len(answers["answer_start"]) == 0:
                        feature["start_position"] = cls_index
                        feature["end_position"] = cls_index
                    else:
                        start_char = answers["answer_start"][0]

                        end_char = start_char + len(answers["text"][0])

                        token_start_index = 0
                        while sequence_ids[token_start_index] != 1:
                            token_start_index += 1

                        token_end_index = len(input_ids) - 1
                        while sequence_ids[token_end_index] != 1:
                            token_end_index -= 1

                        if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                            feature["start_position"] = cls_index
                            feature["end_position"] = cls_index
                        else:
                            while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                                token_start_index += 1
                            feature["start_position"] = token_start_index - 1
                            while offsets[token_end_index][1] >= end_char:
                                token_end_index -= 1
                            feature["end_position"] = token_end_index + 1

                features.append(feature)

        
        features_df = pd.DataFrame(data=features, columns=self.columns)
        return features_df

In [ ]:

trainer = pl.Trainer(
     gpus=1, precision=16
)


In [ ]:

arch_paths = [
        '../input/rembertnd',
    '../input/murillargendv1',
    '../input/xlmrobertalargend701',
    '../input/infoxlmlargend'
#     '../input/rembertcv684',
]

archs_paths_sf = [
        [
      '../input/rembertnd/fold_2_score_0.71650.pth',
     '../input/rembertnd/fold_3_score_0.69884.pth',
    ],
    
    [#'../input/murillargendv1/fold_0_score_0.74931.pth',
    '../input/murillargendv1/fold_1_score_0.70923.pth',
    '../input/murillargendv1/fold_2_score_0.72832.pth',
    '../input/murillargendv1/fold_3_score_0.73089.pth',#,
    '../input/murillargendv1/fold_4_score_0.66972.pth'
    ],
    
    [#'../input/xlmrobertalargend701/fold_0_score_0.722.pth',
    '../input/xlmrobertalargend701/fold_1_score_0.702.pth',
    '../input/xlmrobertalargend701/fold_2_score_0.714.pth',
    '../input/xlmrobertalargend701/fold_3_score_0.713.pth',
    '../input/xlmrobertalargend701/fold_4_score_0.655.pth'
    ],
    [
     #'../input/infoxlmlargend/fold_1_score_0.69163.pth',
     '../input/infoxlmlargend/fold_2_score_0.72072.pth',
     '../input/infoxlmlargend/fold_3_score_0.68688.pth',
        '../input/infoxlmlargend/fold_4_score_0.65945.pth'
    ]
    
#     [
#      # '../input/rembertcv684/fold_2_score_0.709.pth',
#      '../input/rembertcv684/fold_3_score_0.675.pth',
#      '../input/rembertcv684/fold_4_score_0.670.pth'  
#     ]
]
i = 0

for arch_path in arch_paths:
    models_preds = []
    print(arch_path)
    #########################################
    ############## import scripts ###########
    #########################################
    os.chdir(arch_path)
    exec(Path("config.py").read_text())
    exec(Path("dataset.py").read_text())
    exec(Path("model.py").read_text())
#     exec(Path("postprocess.py").read_text())
#     exec(Path('preprocess.py').read_text()) 
    os.chdir('/kaggle/working')
    if "murillarge" in arch_path:
        Default_Config['model_type'] = '../input/muril-large-pt/muril-large-cased'
        Default_Config['model_type_tokenizer'] = '../input/muril-large-pt/muril-large-cased'
    Default_Config['eval_batch_size'] = 128
    if 'rembert' in arch_path:
        Default_Config['eval_batch_size'] = 64

    Config = ConfigTemplate(Default_Config)

        
    ###########################################
    
    tokenizer = torch.load(Path(arch_path) / 'tokenizer.pth')
    preprocessor = FeatureExtractor(tokenizer, cls_token=tokenizer.cls_token, test=True)
    feats = preprocessor.extract_features(test_df)
    test_ds = QADataset(feats, mode='test')
    test_dl = DataLoader(
        test_ds,
        batch_size=Config.eval_batch_size, 
        shuffle=False,
        num_workers=4,
        pin_memory=True, 
        drop_last=False
    )
    
    
    for model_path in archs_paths_sf[i]:
        print('\t', model_path)
        ptmodel = torch.load(model_path, map_location=torch.device('cuda'))
        params = {
            'tokenizer': tokenizer
        }
        model = Model(ptmodel, params=params)
        preds = trainer.predict(model, test_dl)
        sample_id_to_preds = collections.defaultdict(list)

        for batch_indices, batch_start_preds, batch_end_preds in preds:
            for index, feat_start_preds, feat_end_preds in zip(batch_indices, batch_start_preds, batch_end_preds):
                sample_id_to_preds[index].append(normalize((feat_start_preds, feat_end_preds)))

        models_preds.append(sample_id_to_preds)

    open_preds = open(f"preds_{arch_path.split('/')[-1]}.pkl", "wb")
    pickle.dump(models_preds, open_preds)
    
    open_feats = open(f"feats_{arch_path.split('/')[-1]}.pkl", "wb")
    pickle.dump(feats, open_feats)
        
    torch.cuda.empty_cache()
    del ptmodel, model, models_preds, sample_id_to_preds, feats
    gc.collect()
    i+=1


In [ ]:

models_paths = [
        '../input/wechselhindiv1/fold_0_score_0.75536.pth',
        '../input/wechselhindiv1/fold_1_score_0.72663.pth',
        '../input/wechselhindiv1/fold_2_score_0.76238.pth'
]


models_preds = []
arch_path = '../input/wechselhindiv1'
arch_paths.append(arch_path)
print(arch_path)
#########################################
############## import scripts ###########
#########################################
os.chdir(arch_path)
exec(Path("config.py").read_text())
exec(Path("dataset.py").read_text())
exec(Path("model.py").read_text())
#     exec(Path("postprocess.py").read_text())
#     exec(Path('preprocess.py').read_text()) 
os.chdir('/kaggle/working')
Default_Config['eval_batch_size'] = 128
Config = ConfigTemplate(Default_Config)

###########################################

#     tokenizer = torch.load(Path(arch_path) / 'tokenizer.pth')
tokenizer = torch.load('../input/chaiiqa-sampling/roberta-large-wechsel-hindi_tokenizer.pt')
preprocessor = FeatureExtractor(tokenizer, cls_token=tokenizer.cls_token, test=True)

feats = preprocessor.extract_features(test_df.query("language == 'hindi'"))
test_ds = QADataset(feats, mode='test')
test_dl = DataLoader(
    test_ds,
    batch_size=Config.eval_batch_size, 
    shuffle=False,
    num_workers=4,
    pin_memory=True, 
    drop_last=False
)


for model_path in models_paths:
    print('\t', model_path)
    ptmodel = torch.load(model_path, map_location=torch.device('cuda'))
    params = {
        'tokenizer': tokenizer
    }
    model = Model(ptmodel, params=params)
    preds = trainer.predict(model, test_dl)
    sample_id_to_preds = collections.defaultdict(list)

    for batch_indices, batch_start_preds, batch_end_preds in preds:
        for index, feat_start_preds, feat_end_preds in zip(batch_indices, batch_start_preds, batch_end_preds):
            sample_id_to_preds[index].append(normalize((feat_start_preds, feat_end_preds)))

    models_preds.append(sample_id_to_preds)

open_preds = open(f"preds_{arch_path.split('/')[-1]}.pkl", "wb")
pickle.dump(models_preds, open_preds)

open_feats = open(f"feats_{arch_path.split('/')[-1]}.pkl", "wb")
pickle.dump(feats, open_feats)

del ptmodel, model, models_preds, sample_id_to_preds, feats, open_preds, test_ds, test_dl
gc.collect()
torch.cuda.empty_cache()


In [ ]:
models_paths = [
        '../input/wechseltamilv3/fold_0_score_0.59998.pth'
]


models_preds = []
arch_path = '../input/wechseltamilv2'
arch_paths.append(arch_path)
print(arch_path)
#########################################
############## import scripts ###########
#########################################
os.chdir(arch_path)
exec(Path("config.py").read_text())
exec(Path("dataset.py").read_text())
exec(Path("model.py").read_text())
#     exec(Path("postprocess.py").read_text())
#     exec(Path('preprocess.py').read_text()) 
os.chdir('/kaggle/working')
Default_Config['eval_batch_size'] = 128
Config = ConfigTemplate(Default_Config)

###########################################

#     tokenizer = torch.load(Path(arch_path) / 'tokenizer.pth')
tokenizer = torch.load('../input/chaiiqa-sampling/roberta-large-wechsel-tamil_tokenizer.pt')
preprocessor = FeatureExtractor(tokenizer, cls_token=tokenizer.cls_token, test=True)

feats = preprocessor.extract_features(test_df.query("language == 'tamil'"))
test_ds = QADataset(feats, mode='test')
test_dl = DataLoader(
    test_ds,
    batch_size=Config.eval_batch_size, 
    shuffle=False,
    num_workers=4,
    pin_memory=True, 
    drop_last=False
)


for model_path in models_paths:
    print('\t', model_path)
    ptmodel = torch.load(model_path, map_location=torch.device('cuda'))
    params = {
        'tokenizer': tokenizer
    }
    model = Model(ptmodel, params=params)
    preds = trainer.predict(model, test_dl)
    sample_id_to_preds = collections.defaultdict(list)

    for batch_indices, batch_start_preds, batch_end_preds in preds:
        for index, feat_start_preds, feat_end_preds in zip(batch_indices, batch_start_preds, batch_end_preds):
            sample_id_to_preds[index].append(normalize((feat_start_preds, feat_end_preds)))

    models_preds.append(sample_id_to_preds)

open_preds = open(f"preds_{arch_path.split('/')[-1]}.pkl", "wb")
pickle.dump(models_preds, open_preds)

open_feats = open(f"feats_{arch_path.split('/')[-1]}.pkl", "wb")
pickle.dump(feats, open_feats)

del ptmodel, model, models_preds, sample_id_to_preds, feats, open_preds, test_ds, test_dl
gc.collect()
torch.cuda.empty_cache()


In [ ]:

archs_weights = [0.168,0.294,0.441,0.057,0.04,0.04]
assert round(sum(archs_weights[:-1]), 3) == 1.
assert archs_weights[-2] == archs_weights[-1]

archs_preds_files = []
archs_feats_files = []

for arch_path in arch_paths:
    archs_preds_files.append(f"preds_{arch_path.split('/')[-1]}.pkl")
    archs_feats_files.append(f"feats_{arch_path.split('/')[-1]}.pkl")

print(archs_preds_files)
print(archs_feats_files)
new_preds = postprocess_qa_predictions_new(test_df, archs_feats_files, archs_preds_files, archs_weights)

# del archs_feats, archs_preds

In [ ]:
submission_df = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv')
submission_df.PredictionString = submission_df.apply(lambda row: new_preds[row['id']]['text'], axis=1)

test_df = test_df.rename(columns={'index': 'id'})
test_data =pd.merge(left=test_df,right=submission_df,on='id')

In [ ]:
bad_starts = [".", ",", "(", ")", "-", "–",  ",", ";"]
bad_endings = ["...", "-", "(", ")", "–", ",", ";"]

tamil_ad = "கி.பி"
tamil_bc = "கி.மு"
tamil_km = "கி.மீ"
hindi_ad = "ई"
hindi_bc = "ई.पू"


cleaned_preds = []
for pred, context in test_data[["PredictionString", "context"]].to_numpy():
    if pred == "":
        cleaned_preds.append(pred)
        continue
    while any([pred.startswith(y) for y in bad_starts]):
        pred = pred[1:]
    while any([pred.endswith(y) for y in bad_endings]):
        if pred.endswith("..."):
            pred = pred[:-3]
        else:
            pred = pred[:-1]
    
    if any([pred.endswith(tamil_ad), pred.endswith(tamil_bc), pred.endswith(tamil_km), pred.endswith(hindi_ad), pred.endswith(hindi_bc)]) and pred+"." in context:
        pred = pred+"."

    cleaned_preds.append(pred)

test_data["PredictionString"] = cleaned_preds
test_data[['id', 'PredictionString']].to_csv('submission.csv', index=False)

In [ ]:
!head submission.csv